In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForMaskedLM
from tqdm.notebook import tqdm

from bio_if.data.utils import FastaDataset

In [2]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
tokenizer_fn = lambda x: tokenizer(x, return_tensors="pt")['input_ids']

In [3]:
train = FastaDataset('../src/bio_if/data/FLIP/gb1/sampled.fasta', split='train', tokenizer_fn=tokenizer_fn)
val = FastaDataset('../src/bio_if/data/FLIP/gb1/sampled.fasta', split='val', tokenizer_fn=tokenizer_fn)
test = FastaDataset('../src/bio_if/data/FLIP/gb1/sampled.fasta', split='test', tokenizer_fn=tokenizer_fn)

In [4]:
# define an MLP
def init_regressor():
    return nn.Sequential(
        nn.Embedding(tokenizer.vocab_size, 4),
        nn.Flatten(),
        nn.Linear((len(train.seqs[0]) + 2) * 4, 256),
        nn.GELU(),
        nn.Linear(256, 1)
    )

In [5]:
train_dataloader = train.get_dataloader(batch_size=32, shuffle=True, drop_last=True)
val_dataloader = val.get_dataloader(batch_size=32, shuffle=False, drop_last=False)
test_dataloader = test.get_dataloader(batch_size=32, shuffle=False, drop_last=False)

In [6]:
def get_val_loss(model, dataloader, criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids, labels = batch
            outputs = model(input_ids).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    return val_loss / len(dataloader)

In [8]:
EPOCHS = 50
N_VAL_ATTEMPTS = 5
LR = 1e-3

loss_fn = nn.MSELoss()
best_val_loss = float('inf')

val_failures = 0
model = init_regressor()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for batch in tqdm(train_dataloader):
        input_ids, labels = batch
        outputs = model(input_ids).squeeze()
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    print(f'Epoch {epoch} train loss: {train_loss / len(train_dataloader)}')

    val_loss = get_val_loss(model, val_dataloader, loss_fn)
    print(f'Epoch {epoch} val loss: {val_loss}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        val_failures = 0
    else:
        val_failures += 1
        if val_failures >= N_VAL_ATTEMPTS:
            break

  0%|          | 0/196 [00:00<?, ?it/s]

Epoch 0 train loss: 1.4745827712574784
Epoch 0 val loss: 1.2371189919385044


  0%|          | 0/196 [00:00<?, ?it/s]

Epoch 1 train loss: 1.2322396167687006
Epoch 1 val loss: 1.1211298500949687


  0%|          | 0/196 [00:00<?, ?it/s]

...

Epoch 35 train loss: 0.6083347632416657
Epoch 35 val loss: 0.5573306483301249


  0%|          | 0/196 [00:00<?, ?it/s]

Epoch 36 train loss: 0.596706937694428
Epoch 36 val loss: 0.5817333622412248


  0%|          | 0/196 [00:00<?, ?it/s]

Epoch 37 train loss: 0.587603896628229
Epoch 37 val loss: 0.6193950569087808
